In [1]:
import pandas as pd
import glob
import os

In [2]:
# Create dataframes for the brands and the perceptions
df_brands = pd.read_csv("mksci-data/brands.txt", sep="\t", names = ["brand"], header=None)
df_ecos = pd.read_csv("mksci-data\eco\default\jaccard_sqrt.txt", sep=" ", names = ["brand","eco"], header=None)
df_luxury = pd.read_csv("mksci-data\luxury\default\jaccard_sqrt.txt", sep=" ", names = ["brand","luxury"], header=None)
df_nutrition = pd.read_csv("mksci-data/nutrition/default/jaccard_sqrt.txt", sep=" ", names = ["brand","nutrition"], header=None)

In [3]:
# Join the dataframes
df_brands = df_brands.join(df_ecos.set_index('brand'), on='brand')
df_brands = df_brands.join(df_luxury.set_index('brand'), on='brand')
df_brands = df_brands.join(df_nutrition.set_index('brand'), on='brand')

In [4]:
df_brands

,brand,eco,luxury,nutrition
0,abercrombie,0.005557,0.017063,0.002799
1,acura,0.015284,0.034052,0.006515
2,adidasus,0.009700,0.023165,0.005175
3,aeropostale,0.007466,0.016235,0.005335
4,airheads,0.012388,0.014993,0.010786
...,...,...,...,...
163,volvocarsus,0.018135,0.033156,0.007419
164,wheatthins,0.012487,0.018518,0.012382
165,wranglerwestern,0.011097,0.012715,0.007253
166,yoplait,0.017254,0.023589,0.029952


In [5]:
perceptions = ["eco","luxury","nutrition"]
all_files = []
empty_list = []
# Get all the survey files in the folders for every perception
for i in perceptions:
    path = r'C:/Programming/Dashboard_example/mksci-data/' + i + '/survey'
    files = glob.glob(os.path.join(path , "*.txt"))
    all_files += files

# Create a list of dataframes for every perception survey and merge them with the brands dataframe
for i in perceptions:
    for filename in all_files:
        if i in filename:
            df = pd.read_csv(filename, sep="\t", names = ["brand",i], header=None)
            empty_list.append(df)
    df_current = pd.concat([empty_list for empty_list in empty_list if i in empty_list], axis=0)
    df_current.rename(columns={i: i + "_survey"}, inplace=True)
    df_brands = pd.merge(df_brands, df_current, on="brand", how="left")

In [6]:
# Get brand sector
path = r'C:/Programming/Dashboard_example/mksci-data/eco/survey'
files = glob.glob(os.path.join(path , "*.txt"))
sectors = [str.split(str.rsplit(i, sep="\\", maxsplit=1)[1], sep=".", maxsplit=1)[0] for i in files]
sectors


df_sector_whole = pd.DataFrame(columns=["brand","sector"])
for sector, filename in zip(sectors, files):
    print(sector,filename)
    df_sector = pd.read_csv(filename, sep="\t", names = ["brand","sector"], header=None)
    df_sector["sector"] = sector
    df_sector_whole = pd.concat([df_sector_whole, df_sector], axis=0)

apparel C:/Programming/Dashboard_example/mksci-data/eco/survey\apparel.txt
car C:/Programming/Dashboard_example/mksci-data/eco/survey\car.txt
food C:/Programming/Dashboard_example/mksci-data/eco/survey\food.txt
personal_care C:/Programming/Dashboard_example/mksci-data/eco/survey\personal_care.txt


In [7]:
df_brands = pd.merge(df_brands, df_sector_whole, on="brand", how="left")
df_brands = df_brands[df_brands["sector"].notna()]
df_brands.reset_index(inplace=True, drop=True)
df_brands

,brand,eco,luxury,nutrition,eco_survey,luxury_survey,nutrition_survey,sector
0,abercrombie,0.005557,0.017063,0.002799,2.247148,2.731034,NaN,apparel
1,acura,0.015284,0.034052,0.006515,2.744630,2.896458,NaN,car
2,adidasus,0.009700,0.023165,0.005175,2.554307,2.188552,NaN,apparel
3,aeropostale,0.007466,0.016235,0.005335,2.379167,2.244444,NaN,apparel
4,airheads,0.012388,0.014993,0.010786,1.773504,NaN,NaN,food
...,...,...,...,...,...,...,...,...
153,vo5,0.008157,0.016338,0.005879,2.109434,NaN,NaN,personal_care
154,volvocarsus,0.018135,0.033156,0.007419,2.969340,2.766756,NaN,car
155,wheatthins,0.012487,0.018518,0.012382,2.803089,NaN,2.994030,food
156,wranglerwestern,0.011097,0.012715,0.007253,2.675676,1.636364,NaN,apparel


In [8]:
# Rename columns to more appropriate names
df_brands.rename(columns={"eco_survey":"Survey Score Eco-Friendliness", "eco":"Social Perception Score Eco-Friendliness", "luxury_survey":"Survey Score Luxury", 
                          "luxury":"Social Perception Score Luxury", "nutrition_survey":"Survey Score Nutrition", "nutrition":"Social Perception Score Nutrition", "sector":"Sector"}, inplace=True)
df_brands["brand"] = df_brands["brand"].str.capitalize()

In [10]:
df_brands_names = pd.read_csv("mksci-data/df_brands.csv",delimiter=";")
df_brands = df_brands.join(df_brands_names.set_index('brand'), on='brand')

In [11]:
df_brands

,brand,Social Perception Score Eco-Friendliness,Social Perception Score Luxury,Social Perception Score Nutrition,Survey Score Eco-Friendliness,Survey Score Luxury,Survey Score Nutrition,Sector,Brand Name,Brand Text
0,Abercrombie,0.005557,0.017063,0.002799,2.247148,2.731034,NaN,apparel,Abercrombie,AberCr
1,Acura,0.015284,0.034052,0.006515,2.744630,2.896458,NaN,car,Acura,Acura
2,Adidasus,0.009700,0.023165,0.005175,2.554307,2.188552,NaN,apparel,Adidas,Adidas
3,Aeropostale,0.007466,0.016235,0.005335,2.379167,2.244444,NaN,apparel,Aeropostale,AeroP
4,Airheads,0.012388,0.014993,0.010786,1.773504,NaN,NaN,food,Airheads,AHeads
...,...,...,...,...,...,...,...,...,...,...
153,Vo5,0.008157,0.016338,0.005879,2.109434,NaN,NaN,personal_care,VO5,VO5
154,Volvocarsus,0.018135,0.033156,0.007419,2.969340,2.766756,NaN,car,Volvo,Volvo
155,Wheatthins,0.012487,0.018518,0.012382,2.803089,NaN,2.994030,food,Wheat Thins,WThins
156,Wranglerwestern,0.011097,0.012715,0.007253,2.675676,1.636364,NaN,apparel,Wrangler,Wrang


In [12]:
# Save the aggregated dataframe to a csv file
df_brands.to_csv("data/df_brands_aggregated.csv", index=False)